In [1]:
from amacrine_color_schema import *

Connecting sstrauss@172.25.240.205:3306


Load kernels

In [2]:
key_kernel = {
                'experimenter': 'Franke',
                'preprocess_param_set_id': 1,
                'stim_id': 16,
                'noise_id': 1,
                'param_id': 2
            }

kernels = pd.DataFrame.from_dict((BC_ColorKernels() *
                                  BC_ColorKernels.BC_ColorKernels_Green() *
                                  RelativeFieldLocation() *
                                  BC_IplDepth() *
                                  BC_MinMaxQuality() &
                                  key_kernel).fetch(as_dict=True))

Unravel dataframe so that each row is one ROI

In [3]:
columns = ['date', 'exp_num', 'field_id', 'roi',
           'uv_center', 'uv_surround', 
           'green_center', 'green_surround', 
           'stim_uv_center', 'stim_uv_surround',
           'stim_green_center', 'stim_green_surround',
           'traces_uv', 'traces_green',
           'Rel. field location y', 'ipl_depth', 'quality_values']
df = pd.DataFrame(columns=columns)

for index, row in kernels.iterrows():
    
    nb_rois = row['st_kernels_uv'].shape[1]

    current_data = {'date': np.repeat(row['date'], nb_rois), 
                    'exp_num': np.repeat(row['exp_num'], nb_rois), 
                    'field_id': np.repeat(row['field_id'], nb_rois), 
                    'roi': np.arange(1,nb_rois+1),
                    'uv_center': list(row['st_kernels_uv'][:,:,0].T),
                    'uv_surround':  list(row['st_kernels_uv'][:,:,1].T),
                    'green_center':  list(row['st_kernels_green'][:,:,0].T), 
                    'green_surround':  list(row['st_kernels_green'][:,:,1].T),
                    'stim_uv_center': list(np.tile(row['cropped_stimulus_uv'][0,:], (nb_rois,1))),
                    'stim_uv_surround': list(np.tile(row['cropped_stimulus_uv'][1,:], (nb_rois,1))),
                    'stim_green_center': list(np.tile(row['cropped_stimulus_green'][0,:], (nb_rois,1))),
                    'stim_green_surround': list(np.tile(row['cropped_stimulus_green'][1,:], (nb_rois,1))),
                    'traces_uv': list(row['cropped_traces_uv']),
                    'traces_green': list(row['cropped_traces_green']),
                    'Rel. field location y': np.repeat(row['rely'], nb_rois),
                    'ipl_depth': row['depth'],
                    'quality_values': row['quality_values']
                   }
    
    df = pd.concat([df, pd.DataFrame(current_data, columns = columns)], ignore_index = True)

Quality filter ROIs

In [4]:
threshold = 0.1
final_df = df[df['quality_values']>threshold]
final_df = final_df.reset_index(drop=True)

print(np.amin(final_df['quality_values']))
print(len(df) == 3604)
print(len(df) - len(final_df))

0.1051789218557015
True
15


Load Chirp and Local Chirp

In [5]:
key_chirp_g = {
                'experimenter': 'Franke',
                'stim_id': 5,
                'preprocess_param_set_id': 1,
                'resample_f_id': 3
            }
global_chirp = pd.DataFrame.from_dict((NormalizedChirp() & 
                                       key_chirp_g).fetch(as_dict=True))
global_chirp = global_chirp.rename(columns={'time_bin': 'time_bin_global_chirp',
                                            'normalized_chirp': 'global_chirp'})


key_chirp_l = {
                'experimenter': 'Franke',
                'stim_id': 6,
                'preprocess_param_set_id': 1,
                'resample_f_id': 3
                }
local_chirp = pd.DataFrame.from_dict((NormalizedChirp() & 
                                      key_chirp_l).fetch(as_dict=True))
local_chirp = local_chirp.rename(columns={'time_bin': 'time_bin_local_chirp', 
                                          'normalized_chirp': 'local_chirp'})

Drop unnecessary columns

In [6]:
drop = ['experimenter', 'stim_id', 'presentation_id', 'preprocess_param_set_id', 'resample_f_id']

global_chirp = global_chirp.drop(drop, axis = 1)
local_chirp = local_chirp.drop(drop, axis = 1)

Merge different dataframes

In [7]:
match_on = ['date', 'exp_num', 'field_id', 'roi']

interm = final_df.merge(global_chirp, how='inner', on=match_on)
all_data = interm.merge(local_chirp, how='inner', on=match_on)

assert(len(final_df) == len(all_data))

Add batch, mouse, eye to the dataframe

In [8]:
grouped_df = all_data.groupby(['date', 'exp_num', 'field_id'])
field_ID = -999*np.ones(len(all_data))
current_batch = 0
for name, group in grouped_df:
    group_indices = group.index.to_numpy()
    field_ID[group_indices] = current_batch
    current_batch = current_batch + 1
all_data['field'] = field_ID

grouped_df = all_data.groupby(['date', 'exp_num'])
eye_ID = -999*np.ones(len(all_data))
current_eye = 0
for name, group in grouped_df:
    group_indices = group.index.to_numpy()
    eye_ID[group_indices] = current_eye
    current_eye = current_eye + 1
all_data['eye'] = eye_ID

grouped_df = all_data.groupby(['date'])
mouse_ID = -999*np.ones(len(all_data))
current_mouse = 0
for name, group in grouped_df:
    group_indices = group.index.to_numpy()
    mouse_ID[group_indices] = current_mouse
    current_mouse = current_mouse + 1
all_data['mouse'] = mouse_ID

In [9]:
print(np.unique(all_data['field']).shape)

(21,)


One extra time bin for one ROI

In [10]:
print(len(all_data['local_chirp'].iloc[1229]))
print(all_data['time_bin_local_chirp'].iloc[1229])
all_data.at[1229, 'time_bin_local_chirp'] = all_data['time_bin_local_chirp'].iloc[1229][0:-1]
all_data.at[1229, 'local_chirp'] = all_data['local_chirp'].iloc[1229][0:-1]
print(len(all_data['local_chirp'].iloc[1229]))

332
[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7
  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1
  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5
  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9
  7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3
  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7
  9.8  9.9 10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9 11.  11.1
 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12.  12.1 12.2 12.3 12.4 12.5
 12.6 12.7 12.8 12.9 13.  13.1 13.2 13.3 13.4 13.5 13.6 13.7 13.8 13.9
 14.  14.1 14.2 14.3 14.4 14.5 14.6 14.7 14.8 14.9 15.  15.1 15.2 15.3
 15.4 15.5 15.6 15.7 15.8 15.9 16.  16.1 16.2 16.3 16.4 16.5 16.6 16.7
 16.8 16.9 17.  17.1 17.2 17.3 17.4 17.5 17.6 17.7 17.8 17.9 18.  18.1
 18.2 18.3 18.4 18.5 18.6 18.7 18.8 18.9 19.  19.1 19.2 19.3 19.4 19.5
 1

Check time bins

In [11]:
referenceA = all_data['time_bin_local_chirp'].iloc[0]
referenceB = all_data['time_bin_global_chirp'].iloc[0]
assert np.all(referenceA == referenceB)

for index, row in all_data.iterrows():
    assert np.all(referenceA == row['time_bin_local_chirp'])
    assert np.all(referenceB == row['time_bin_global_chirp'])

Save dataframe

In [12]:
#all_data.to_pickle('Data/BCs/Dataframes/control_data.pkl')